# DSCI 100 Group Project

### Introduction
To do: 
- Write a bit about what the data set is (couple sentences about PLAICraft, purpose of the study)
- State the question: demand forecasting
- Describe the dataset used: sessions AND players

New question: predicting subscription status
- Use quantitative variables (ignore experience)
- Can use age, hours played, number of sessions, and session length: the last two use some more interesting wrangling techniques to figure out that can show our knowledge and make our investigation a bit different
- Call it "playing behaviour"?

The data from the players.csv dataset contains the following 7 variables with 196 players recorded:

-experience levels (from Pro to Beginner recorded as characters)

-if they are a subscriber (true or false recorded as logical)

-their email, amount of played hours on the game (recorded as double)

-their name (recorded as characters)

-gender (Female, Male, Non-binary, and more, all recorded as characters)

-age (in years recorded as double).


### Exploratory Data Analysis

To do: 
- Split the data into a training set and a testing set. Only explore the training set.
- Calculate basic summary statistics on time played and make a few visualizations, like in planning document